In [1]:
import rioxarray
import rasterio
import pandas as pd
import os
import geopandas
import ftplib
import re
import zipfile
from tqdm import tqdm

In [2]:
data_input_dir = "data/gpm-imerg"
input_raster = os.path.join(data_input_dir, "3B-MO-GIS.MS.MRG.3IMERG.20210101-S000000-E235959.01.V06B.total.accum.tif")
aoi_dir = "data/aoi.geojson"
geodf = geopandas.read_file(aoi_dir)

In [3]:
geodf.bounds

,minx,miny,maxx,maxy
0,18.406925,50.476079,19.680133,51.179343


In [4]:
gpm_imerg_zip_dir = "data/gpm-imerg/zip"
gpm_imerg_extracted_dir = "data/gpm-imerg/extracted"
zips = os.listdir(gpm_imerg_zip_dir)
file_pattern = ".*.zip"
file_patter = re.compile(file_pattern)
zips = list(filter(file_patter.match, zips))
for zipf in zips:
    try:
        with zipfile.ZipFile(os.path.join(gpm_imerg_zip_dir, zipf), 'r') as zip_ref:
            zip_ref.extractall(gpm_imerg_extracted_dir)
    except:
        print("Error in: " + zipf)
        

Error in: 3B-MO-L.GIS.IMERG.20190601.V06B.zip


In [5]:
gpm_imerg_extracted_dir = "data/gpm-imerg/extracted"
files = os.listdir(gpm_imerg_extracted_dir)
file_pattern = ".*V06B.tif"
file_patter = re.compile(file_pattern)
file_list = list(filter(file_patter.match, files))
#file_list

In [6]:
data = rasterio.open(input_raster)

In [7]:
data = rioxarray.open_rasterio(data)[0]

In [8]:
data = data.rio.write_crs("EPSG:4326")

In [9]:
data = data.astype("float32")

In [10]:
data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)

In [42]:
data = data.rio.write_crs("EPSG:4326")

In [11]:
df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(gpm_imerg_extracted_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data = data.rio.write_crs("EPSG:4326")
    masked = data.to_masked_array()
    datetime = f[18:22] + "-" + f[22:24] + "-" + f[24:26]
    #df = df.append({'date': datetime, 'value': masked.data, 'mask': masked.mask}, ignore_index=True)
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)

/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_15856/2969114692.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_15856/2969114692.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_15856/2969114692.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
/var/folders/4p/ctysll0s0551b50jtg8djbs40000gp/T/ipykernel_15856/2969114692.py:14: FutureWarning: The frame.append met

In [12]:
df.iloc[0].value.shape

(8, 13)

In [13]:
df.to_pickle("data/gpm-imerg_df.pickle")

In [15]:
df.iloc[0]

date                                            2019-12-01
value    [[nan, nan, nan, nan, nan, nan, 18.0, 23.0, 23...
Name: 0, dtype: object

In [14]:
host = "jsimpsonftps.pps.eosdis.nasa.gov"
port = 21
login = "pawel.korytowski@outlook.com"
data_path = "/data/imerg/gis"

def establish_conn():
    session = ftplib.FTP_TLS()
    session.connect(host, port)
    session.login(user = login, passwd = login)
    return session

session = establish_conn()
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

file_pattern = "3B-MO*.zip"
file_pattern = re.compile(file_pattern)

#for year in range(2002, 2023, 1):
#    for month in months:
#        file_path = data_path + "/" + str(year) + "/" + month
file_path = os.path.join(data_path, "2002", "01")
session.cwd(file_path)
files = session.nlst()
file = list(filter(file_pattern.match, files))
file

KeyboardInterrupt: 